In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
from sklearn.metrics import f1_score                                                                                                                                                                

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchtext
import nltk
import time
from datetime import timedelta
import numpy as np
from sklearn import metrics
from nltk.corpus import stopwords
import re
from imblearn.over_sampling import SMOTE #upsampleing SMOTE
from sklearn.metrics import precision_recall_fscore_support as score
from torchsummary import summary 



train_path = 'input/start-kit/offenseval-training-vB.csv'
test_path = 'input/testB/testset-taskb.tsv'
embed_path = 'input/embedding/glove.840B.300d.txt'
submission_path = 'input/submissionB.csv'
model_path = 'model/task2_model.pkl'
if torch.cuda.is_available():
    DEVICE = 'cuda:0'
else:
    DEVICE = 'cpu'

batch_size = 128

def save_model(model, model_path):
    """Save model."""
    torch.save(model.state_dict(), model_path)


def load_model(model, model_path, use_cuda=False):
    """Load model."""
    map_location = 'cpu'
    if use_cuda and torch.cuda.is_available():
        map_location = 'cuda:0'
    model.load_state_dict(torch.load(model_path, map_location))
    return model

stopWords = set(stopwords.words('english'))

In [3]:
stopWords.add('@USER')
stopWords.add('#')

In [4]:
# y_pred = torch.randn((32,3))
# print(y_pred)
# y_pred = torch.max(y_pred, 1)[1].data.cpu().numpy()
# print(y_pred)

In [5]:
# #labels change to int,for taskA
# path = 'input/start-kit/offenseval-training-v1.tsv'
# train=pd.read_csv(path, sep='\t', header=0)
# class_mapping1 = {'NOT':0, 'OFF':1}
# class_mapping2 = {'UNT':0, 'TIN':1}
# class_mapping3 = {'IND':0, 'GRP':1,'OTH':2}
# train['subtask_a'] = train['subtask_a'].map(class_mapping1)
# train['subtask_b'] = train['subtask_b'].map(class_mapping2)
# train['subtask_c'] = train['subtask_c'].map(class_mapping3)
# train[['subtask_b', 'subtask_c']] = train[['subtask_b', 'subtask_c']].fillna(-1)
# train[['subtask_b', 'subtask_c']] = train[['subtask_b', 'subtask_c']].astype(int, errors='ignore')
# train.to_csv('input/start-kit/offenseval-training-vA.csv',index=0) #不保存行索引

# #for taskB
# path = 'input/start-kit/offenseval-training-vA.csv'
# train=pd.read_csv(path, header=0)
# taskB_train = train.loc[train['subtask_a'] == 1]
# taskB_train.to_csv('input/start-kit/offenseval-training-vB.csv',index=0) #不保存行索引

# #for taskC
# path = 'input/start-kit/offenseval-training-vB.csv'
# train=pd.read_csv(path, header=0)
# taskC_train = train.loc[train['subtask_b'] == 1]
# taskC_train.to_csv('input/start-kit/offenseval-training-vC.csv',index=0) #不保存行索引

path = 'input/start-kit/offenseval-training-vA.csv'
train=pd.read_csv(path, header=0)
print(train.groupby(by='subtask_a').size())


path = 'input/start-kit/offenseval-training-vB.csv'
train=pd.read_csv(path, header=0)
print(train.groupby(by='subtask_b').size())

path = 'input/start-kit/offenseval-training-vC.csv'
train=pd.read_csv(path, header=0)
print(train.groupby(by='subtask_c').size())

subtask_a
0    8840
1    4400
dtype: int64
subtask_b
0     524
1    3876
dtype: int64
subtask_c
0    2407
1    1074
2     395
dtype: int64


In [6]:
def sample_balance(X, y):
    '''
    :param X: input
    :param y: label
    :return: balacend X,y
    '''
    model_smote = SMOTE() 
    x_smote_resampled, y_smote_resampled = model_smote.fit_resample(X, y) 
    return x_smote_resampled, y_smote_resampled 


In [7]:
def pre():
    """Pre-process model."""

    print("Pre-processing...")

    # load data
    fix_length = 48
    text = torchtext.data.Field(
        sequential=True, use_vocab=True, lower=True,
        tokenize=nltk.word_tokenize, batch_first=True,
        is_target=False, fix_length=fix_length)
    
    
    
    targetA = torchtext.data.Field(
        sequential=False, use_vocab=False,
        batch_first=True, is_target=True)
    
    targetB = torchtext.data.Field(
    sequential=False, use_vocab=False,
    batch_first=True, is_target=True)

    targetC = torchtext.data.Field(
    sequential=False, use_vocab=False,
    batch_first=True, is_target=True)
    
    
    dataset = torchtext.data.TabularDataset(
        train_path, format='csv',
        fields={
                "tweet": ('text', text),
#                 "subtask_a": ('target', targetA),
                "subtask_b": ('target', targetB),
#                 "subtask_c": ('target', targetC),
               }
    )
    
    data_test = torchtext.data.TabularDataset(
        test_path, format='tsv',
        fields={
                "tweet": ('text', text)
               }
    )

    
    # build vocab
    text.build_vocab(dataset, data_test, min_freq=3)
    
#  show the frequency
#     vocab = text.vocab
#     print(vocab.freqs)
    
    
    text.vocab.load_vectors(torchtext.vocab.Vectors(embed_path))
    vocab_size = len(text.vocab.itos)
    padding_idx = text.vocab.stoi[text.pad_token]

    # split data
    data_train, data_val = dataset.split(split_ratio=0.9)
    
    def clean(example, vocab):
        return [word for word in example if word in vocab and word not in stopWords and word.isalpha()]
    # train here is the `Dataset` instance obtained through .splits()
    for example in data_test:
      # assuming field name is 'text'
        textFiltered = clean(example.text, text.vocab.stoi)
        example.text = textFiltered
    
    print("train set size:", len(data_train))
    print("val set size:", len(data_val))
    print("test set size:", len(data_test))
    print("vocab size:", len(text.vocab.itos))
    print("embed shape:", text.vocab.vectors.shape)
    print('')

    # # save data
    # save_pickle(data_train, pickle_path, 'data_train.pkl')
    # save_pickle(data_val, pickle_path, 'data_val.pkl')
    # print('')

    args_dict = {
        "data_train": data_train, "data_val": data_val,
        "data_test": data_test, "vocab_size": vocab_size,
        "padding_idx": padding_idx,"all_data":dataset}

    return args_dict

In [8]:
args = pre()

Pre-processing...
train set size: 3960
val set size: 440
test set size: 240
vocab size: 3357
embed shape: torch.Size([3357, 300])



In [9]:
all_data = args['all_data']
print(all_data)

In [10]:
train_iter = torchtext.data.Iterator(
    dataset=args['data_train'], batch_size=128,
    train=False, device=DEVICE, sort=False)

valid_iter = torchtext.data.Iterator(
    dataset=args['data_val'], batch_size=128,
    train=False, device=DEVICE, sort=False)

text_list = []
target_list = []

for batch in train_iter:
    text = batch.text
    target = batch.target
    text_list.append(text.cpu().numpy())
    target_list.append(target.cpu().numpy())
    
for batch in valid_iter:
    text = batch.text
    target = batch.target
    text_list.append(text.cpu().numpy())
    target_list.append(target.cpu().numpy())
    
text_list = np.concatenate(text_list, axis=0)
target_list = np.concatenate(target_list, axis=0)

b_X,b_y = sample_balance(text_list, target_list)
from collections import Counter

In [11]:
class TextCNN(nn.Module):
    """Text classification model by character CNN, the implementation of paper
    'Yoon Kim. 2014. Convolution Neural Networks for Sentence
    Classification.'
    """

    def __init__(self, args):
        super(TextCNN, self).__init__()

        vocab_size = args["vocab_size"]
        pretrained_embed = args["pretrained_embed"]
        padding_idx = args["padding_idx"]
        self.is_mul =args['is_mul']
        num_classes = 1
        mul_num_classes = 3
        kernel_nums = [100, 100, 100]
        kernel_sizes = [3, 4, 5]
        embed_dim = 300
        hidden_dim = 100
        drop_prob = 0.5

        # no support for pre-trained embedding currently
        if pretrained_embed is None:
            self.embed = nn.Embedding(vocab_size, embed_dim)
        else:
            self.embed = nn.Embedding.from_pretrained(
                pretrained_embed, freeze=False)
        self.embed.padding_idx = padding_idx
        self.convs = nn.ModuleList(
            [nn.Conv1d(embed_dim, kn, ks)
             for kn, ks in zip(kernel_nums, kernel_sizes)])
        self.fc = nn.Linear(sum(kernel_nums), hidden_dim)
        self.drop = nn.Dropout(drop_prob)
        
        if not self.is_mul:
            self.out = nn.Linear(hidden_dim, num_classes)
            self.loss = nn.BCEWithLogitsLoss()
        else:
            self.out = nn.Linear(hidden_dim, mul_num_classes)
            self.loss = nn.CrossEntropyLoss()


    def forward(self, word_seq):
        # embed
        e = self.drop(self.embed(word_seq))  # [b,msl]->[b,msl,e]

        # conv and pool, [b,msl,e]->[b,h,msl]
        e = e.transpose(1, 2)  # [b,msl,e]->[b,e,msl]
        ps = []
        for conv in self.convs:
            c = conv(e)  # [b,e,msl]->[b,h,msl-k]
            p = F.max_pool1d(c, kernel_size=c.size(-1)).squeeze(-1)  # [b,h]
            ps.append(p)
        p = torch.cat(ps, dim=1)  # [b,h]

        # feed-forward, [b,h]->[b]
        f = self.drop(self.fc(p))
        if not self.is_mul:
            logits = self.out(f).squeeze(-1) # b
        else:
            logits = self.out(f)# b x mul_class

        return logits

In [12]:
# precision, recall, fscore, support = score(y_test, predicted)

# predicted = [1,2,3,4,5,1,2,1,1,4,5] 
# y_test = [1,2,3,4,5,1,2,1,1,4,1]
# print('precision: {}'.format(precision))
# print('recall: {}'.format(recall))
# print('fscore: {}'.format(fscore))
# print('support: {}'.format(support))

### training

In [13]:
def evaluate(predict, truth, threshold=0.33,average = True,metric= 'macro'):
    """Compute evaluation metrics.

    :param predict: list of Tensor
    :param truth: list of dict
    :param threshold: threshold of positive probability
    :return eval_results: dict, format {name: metrics}.
    """
    y_trues, y_preds = [], []
#     print('predict:',predict)
    for y_true, logit in zip(truth, predict):
        y_pred = (torch.sigmoid(logit) > threshold).long().cpu().numpy()
        y_true = y_true.cpu().numpy()
        y_trues.append(y_true)
        y_preds.append(y_pred)
    y_true = np.concatenate(y_trues, axis=0)
    y_pred = np.concatenate(y_preds, axis=0)
    
#     print(y_true)
#     print(y_pred)
    
#     precision = metrics.precision_score(y_true, y_pred, pos_label=1)
#     recall = metrics.recall_score(y_true, y_pred, pos_label=1)
#     f1 = metrics.f1_score(y_true, y_pred, pos_label=1)
    if average:
        precision = metrics.precision_score(y_true, y_pred, average = metric)
        recall = metrics.recall_score(y_true, y_pred,average = metric)
        f1 = metrics.f1_score(y_true, y_pred, average = metric)
    else:
        precision, recall, f1, support = score(y_true, y_pred)
    metrics_dict = {"precision": precision, "recall": recall, "f1": f1}
    
    return metrics_dict


def multi_evaluate(predict, truth,average = True):
    """Compute evaluation metrics.

    :param predict: list of Tensor
    :param truth: list of dict
    :param threshold: threshold of positive probability
    :return eval_results: dict, format {name: metrics}.
    """
    y_trues, y_preds = [], []
    for y_true, logit in zip(truth, predict):
        
        softmax = nn.Softmax(1)# softmax on each row
        y_pred = softmax(logit)
        y_pred = torch.max(y_pred, 1)[1].data.cpu().numpy()
        y_true = y_true.cpu().numpy()
        y_trues.append(y_true)
        y_preds.append(y_pred)
    y_true = np.concatenate(y_trues, axis=0)
    y_pred = np.concatenate(y_preds, axis=0)
    
#     precision = metrics.precision_score(y_true, y_pred, pos_label=1)
#     recall = metrics.recall_score(y_true, y_pred, pos_label=1)
#     f1 = metrics.f1_score(y_true, y_pred, pos_label=1)

    if average:
        precision = metrics.precision_score(y_true, y_pred, average = 'macro')
        recall = metrics.recall_score(y_true, y_pred,average = 'macro')
        f1 = metrics.f1_score(y_true, y_pred, average = 'macro')
    else:
        precision, recall, f1, support = score(y_true, y_pred)
        
    metrics_dict = {"precision": precision, "recall": recall, "f1": f1}

    return metrics_dict



def print_eval_results(results):
    """Override this method to support more print formats.
    :param results: dict, (str: float) is (metrics name: value)
    """
    return ", ".join(
        [str(key) + "=" + "{:.4f}".format(value)
         for key, value in results.items()])


def test(network, data_iter, threshold=0.33,average = True,metric = 'macro'):
    # transfer model to gpu if available
    network = network.to(DEVICE)

    # turn on the testing mode; clean up the history
    network.eval()
    output_list = []
    truth_list = []
    
    # predict
    for batch in data_iter:
        text, target = batch.text, batch.target
        text = text.cuda()

        with torch.no_grad():
            prediction = network(text)

        output_list.append(prediction.detach())
        truth_list.append(target.detach())
        
    # evaluate
    eval_results = evaluate(output_list, truth_list, threshold,average,metric)
#     eval_results = multi_evaluate(output_list, truth_list,average)
    if average:
        print("[tester] {}".format(print_eval_results(eval_results)))

    return eval_results


In [17]:
def train(epoch, model, loss_func, optimizer, train_iter,valid_iter,show_every = 1000):
    start = time.time()
    best_f1_score = 0
    for e in range(epoch):
        model.train()
        for step,batch in enumerate(train_iter):
            text, target = batch.text, batch.target
            text = text.cuda()
            optimizer.zero_grad()
#             print(text.shape)
            logits = model(text)
#             print(logits)
#             print(target)
            loss = model.loss(logits, target.float())
#             loss = model.loss(logits, target)
            loss.backward()
            optimizer.step()
            
            if step % show_every == 0:
                end = time.time()
                diff = timedelta(seconds=round(end - start))
                print_output = "[epoch: {:>3} step: {:>4}]" \
                    " train loss: {:>4.6} time: {}".format(
                        e, step, loss.item(), diff)
                print(print_output)
        
        
        #show F1 on validation and save model if it is the best model so far
        eval_results = test(model, valid_iter)
        if eval_results['f1'] > best_f1_score:
            best_f1_score = eval_results['f1']
            save_model(model, model_path)
            print("**Saved better model selected by validation.")

### hyperparameter tuning

In [18]:
# args = pre()

In [19]:
data_train = args["data_train"]
data_val = args["data_val"]
pretrained_embed = data_train.fields["text"].vocab.vectors

# define model
model_args = {
    "vocab_size": args["vocab_size"],
    "padding_idx": args["padding_idx"],
    "pretrained_embed": pretrained_embed,
    "is_mul":False
}


# define batch iterator
train_iter = torchtext.data.Iterator(
    dataset=data_train, batch_size=batch_size,
    train=True, shuffle=True, sort=False,
    device=DEVICE)

# define batch iterator
valid_iter = torchtext.data.Iterator(
    dataset=data_val, batch_size=batch_size,
    train=False, device=DEVICE, sort=False)

model = TextCNN(model_args).cuda()
# print(model)
loss_func = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4,betas=(0.5, 0.999))


train(100,model,loss_func,optimizer,train_iter,valid_iter)

[epoch:   0 step:    0] train loss: 0.608141 time: 0:00:00


/home/fl4918/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/fl4918/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[tester] precision=0.4398, recall=0.5000, f1=0.4680
**Saved better model selected by validation.
[epoch:   1 step:    0] train loss: 0.484319 time: 0:00:01
[tester] precision=0.4398, recall=0.5000, f1=0.4680
[epoch:   2 step:    0] train loss: 0.422223 time: 0:00:02
[tester] precision=0.4398, recall=0.5000, f1=0.4680
[epoch:   3 step:    0] train loss: 0.400204 time: 0:00:02
[tester] precision=0.4398, recall=0.5000, f1=0.4680
[epoch:   4 step:    0] train loss: 0.350895 time: 0:00:03
[tester] precision=0.4398, recall=0.5000, f1=0.4680
[epoch:   5 step:    0] train loss: 0.363688 time: 0:00:04
[tester] precision=0.4398, recall=0.5000, f1=0.4680
[epoch:   6 step:    0] train loss: 0.392987 time: 0:00:04
[tester] precision=0.4398, recall=0.5000, f1=0.4680
[epoch:   7 step:    0] train loss: 0.314992 time: 0:00:05
[tester] precision=0.4398, recall=0.5000, f1=0.4680
[epoch:   8 step:    0] train loss: 0.364011 time: 0:00:06
[tester] precision=0.4398, recall=0.5000, f1=0.4680
[epoch:   9 ste

[tester] precision=0.5568, recall=0.5154, f1=0.5086
[epoch:  72 step:    0] train loss: 0.0505355 time: 0:00:49
[tester] precision=0.5597, recall=0.5209, f1=0.5189
[epoch:  73 step:    0] train loss: 0.0629438 time: 0:00:49
[tester] precision=0.5679, recall=0.5278, f1=0.5295
[epoch:  74 step:    0] train loss: 0.0207552 time: 0:00:50
[tester] precision=0.5562, recall=0.5252, f1=0.5263
[epoch:  75 step:    0] train loss: 0.054356 time: 0:00:51
[tester] precision=0.5471, recall=0.5184, f1=0.5160
[epoch:  76 step:    0] train loss: 0.0316518 time: 0:00:51
[tester] precision=0.5417, recall=0.5171, f1=0.5145
[epoch:  77 step:    0] train loss: 0.0244399 time: 0:00:52
[tester] precision=0.5471, recall=0.5184, f1=0.5160
[epoch:  78 step:    0] train loss: 0.043176 time: 0:00:53
[tester] precision=0.5618, recall=0.5265, f1=0.5279
[epoch:  79 step:    0] train loss: 0.0393303 time: 0:00:54
[tester] precision=0.5471, recall=0.5184, f1=0.5160
[epoch:  80 step:    0] train loss: 0.0294195 time: 0:

In [20]:
# define model
model_args = {
    "vocab_size": args["vocab_size"],
    "padding_idx": args["padding_idx"],
    "pretrained_embed": None,
    "is_mul":False
}
model = TextCNN(model_args)
load_model(model, model_path, use_cuda=True)

def parmTuning(args,valid_iter):
    # test and threshold selection
    data_val = args["data_val"]
    best_thresh, best_f1 = 0., 0.
    for thresh in np.arange(0.1, 0.501, 0.01):
        thresh = np.round(thresh, 2)
        f1 = test(model, valid_iter, threshold=thresh,average = True)["f1"]
        print("threshold: {:>.2f} f1: {:>.4f}".format(thresh, f1))
        if f1 > best_f1:
            best_thresh, best_f1 = thresh, f1

    args["threshold"] = best_thresh


    print("best f1 on dev: {:>.4f} threshold: {:>.2f}".format(
        best_f1, best_thresh))
    
        
    metrics = test(model, valid_iter, threshold=best_thresh,average = False)
    print(metrics)
    
    return args

In [ ]:
# 
args = parmTuning(args,valid_iter)

/home/fl4918/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/fl4918/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[tester] precision=0.4398, recall=0.5000, f1=0.4680
threshold: 0.10 f1: 0.4680
[tester] precision=0.4398, recall=0.5000, f1=0.4680
threshold: 0.11 f1: 0.4680
[tester] precision=0.4398, recall=0.5000, f1=0.4680
threshold: 0.12 f1: 0.4680
[tester] precision=0.4396, recall=0.4987, f1=0.4673
threshold: 0.13 f1: 0.4673
[tester] precision=0.4396, recall=0.4987, f1=0.4673
threshold: 0.14 f1: 0.4673
[tester] precision=0.4395, recall=0.4974, f1=0.4667
threshold: 0.15 f1: 0.4667
[tester] precision=0.4395, recall=0.4974, f1=0.4667
threshold: 0.16 f1: 0.4667
[tester] precision=0.4395, recall=0.4974, f1=0.4667
threshold: 0.17 f1: 0.4667
[tester] precision=0.4395, recall=0.4974, f1=0.4667
threshold: 0.18 f1: 0.4667
[tester] precision=0.4395, recall=0.4974, f1=0.4667
threshold: 0.19 f1: 0.4667
[tester] precision=0.4395, recall=0.4974, f1=0.4667
threshold: 0.20 f1: 0.4667
[tester] precision=0.4395, recall=0.4974, f1=0.4667
threshold: 0.21 f1: 0.4667
[tester] precision=0.6072, recall=0.5068, f1=0.4851


In [ ]:
args

In [ ]:
class Predictor(object):
    """An interface for predicting outputs based on trained models.
    """

    def __init__(self, batch_size=8, use_cuda=False):
        self.batch_size = batch_size
        self.use_cuda = use_cuda
        self.device = 'cpu'
        if torch.cuda.is_available() and self.use_cuda:
            self.device = 'cuda:0'

    def predict(self, network, data, threshold=0.33,mul = False):
        # transfer model to gpu if available
        network = network.to(self.device)

        # turn on the testing mode; clean up the history
        network.eval()
        batch_output = []

        # define batch iterator
        data_iter = torchtext.data.Iterator(
            dataset=data, batch_size=self.batch_size,
            train=False, device=self.device, sort=False)
    
        for batch in data_iter:
            text = batch.text
            with torch.no_grad():
                prediction = network(text)
            batch_output.append(prediction.detach())

        if mul:
            return self.softmax_processor(batch_output)
        else:
            return self._post_processor(batch_output, threshold)

    def _post_processor(self, batch_output, threshold=0.33):
        """Convert logit tensor to label."""
        y_preds = []
        for logit in batch_output:
            y_pred = (torch.sigmoid(logit) > threshold).long().cpu().numpy()
            y_preds.append(y_pred)
        y_pred = np.concatenate(y_preds, axis=0)

        return y_pred
    
    
    def softmax_processor(self,batch_output):
        y_preds = []
        for logit in batch_output:
            softmax = nn.Softmax(1)# softmax on each row
            y_pred = softmax(logit)
            y_pred = torch.max(y_pred, 1)[1].data.cpu().numpy()
            y_preds.append(y_pred)
        y_pred = np.concatenate(y_preds, axis=0)
        return y_pred


In [ ]:
# define predictor
predictor = Predictor(batch_size=128, use_cuda=True)

# define model
model_args = {
    "vocab_size": args["vocab_size"],
    "padding_idx": args["padding_idx"],
    "pretrained_embed": None,
    "is_mul":False
}
model = TextCNN(model_args)
load_model(model, model_path, use_cuda=True)


# predict
data_test = args["data_test"]
threshold = args["threshold"]
#single:
# y_pred = predictor.predict(model, data_test, threshold=threshold,mul = False)
#mul
y_pred = predictor.predict(model, data_test, threshold=threshold,mul = True)

In [ ]:
# submit result
test_df = pd.read_csv(test_path, sep='\t', index_col=False, header=0)
data = {"qid": test_df["id"],"prediction": y_pred}
print(y_pred)
# print(data)
subm_df = pd.DataFrame(data=data)

data = subm_df['prediction']

# class_mapping1 = {'NOT':0, 'OFF':1}
# data.loc[data == 0] = 'NOT'
# data.loc[data == 1] = 'OFF'




# # class_mapping2 = {'UNT':0, 'TIN':1}

data.loc[data == 0] = 'UNT'
data.loc[data == 1] = 'TIN'




# data.loc[data == 0] = 'IND'
# data.loc[data == 1] = 'GRP'
# data.loc[data == 2] = 'OTH'
subm_df.to_csv(submission_path, header=False, index=False)
print("submission saved as {}.".format(submission_path))
print('')

In [ ]:
# define predictor
predictor = Predictor(batch_size=128, use_cuda=True)

# define model
model_args = {
    "vocab_size": args["vocab_size"],
    "padding_idx": args["padding_idx"],
    "pretrained_embed": None,
    "is_mul":False
}
model = TextCNN(model_args)
load_model(model, model_path, use_cuda=True)
model = model.cuda()
model = model.eval()
y_preds = []
real_y = []
batch_output = []


all_data_iter = torchtext.data.Iterator(
    dataset=args['all_data'], batch_size=128,
    train=False, device=DEVICE, sort=False)

for batch in all_data_iter:
    text = batch.text
    target = batch.target
    text = text.cuda()
    prediction = model(text)
    real_y.append(target)
    batch_output.append(prediction.detach())
    
for logit in batch_output:
    #y_pred = (torch.sigmoid(logit) > threshold).long().cpu().numpy()
    softmax = nn.Softmax(1)# softmax on each row
    y_pred = softmax(logit)
    y_pred = torch.max(y_pred, 1)[1].data.cpu().numpy()
    y_preds.append(y_pred)
y_pred = np.concatenate(y_preds, axis=0)
real_y = np.concatenate(real_y, axis=0)
print('y_pred:',y_pred)
print(Counter(y_pred))
print('real_y:',real_y)
print(Counter(real_y))
print( (y_pred == real_y).sum()/ len(real_y))


